In [1]:
# Text text processing library
import torchtext
from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import itertools as it
# from models import *
# from helpers import *
# import main
import matplotlib.pyplot as plt
import spacy
import time
MAX_LEN = 20
MIN_FREQ = 5

In [2]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

BOS_WORD = '<s>'
EOS_WORD = '</s>'
DE = data.Field(tokenize=tokenize_de)

# only target needs BOS/EOS:
EN = data.Field(tokenize=tokenize_en, init_token = BOS_WORD, eos_token = EOS_WORD) 

train, val, test = datasets.IWSLT.splits(exts=('.de', '.en'), fields=(DE, EN), 
                                         filter_pred=lambda x: len(vars(x)['src']) <= MAX_LEN and 
                                         len(vars(x)['trg']) <= MAX_LEN)

In [3]:
DE.build_vocab(train.src, min_freq=MIN_FREQ)
EN.build_vocab(train.trg, min_freq=MIN_FREQ)

pred_set = []
for i, line in enumerate(open("source_test.txt"), 1):
    words = line.split()[:-1]
    pred_set.append([DE.vocab.stoi[s] for s in words])

train_iter, val_iter, test_iter = data.BucketIterator.splits((train, val, test), batch_size=32, device=-1,
                                                  repeat=False, sort_key=lambda x: len(x.src))

In [17]:
batch = next(iter(test_iter))
# sent_inspect(batch,4)
def sent_inspect(batch, idx=0):
    print("Source")
    print(' '.join([DE.vocab.itos[w] for w in batch.src.data[:,idx]]))
    print("Target")
    print(' '.join([EN.vocab.itos[w] for w in batch.trg.data[:,idx]]))
# print(DE.vocab.stoi['<pad>'])

In [81]:
for i in range(10):
    print(' '.join(DE.vocab.itos[w] for w in debug_set[i]))
    print(' '.join(EN.vocab.itos[w] for w in debug_ans[i]))    

wo sich der Lauf des <unk> früher befand .
<s> We started to be able to see where the <unk> used to flow . </s> <pad> <pad>
Das führte mich dazu , Satellitenbilder zu benutzen .
<s> This is really what brought me to using satellite imagery . </s> <pad> <pad> <pad> <pad> <pad>
Wie unterscheidet sich diese Geschichte von der anderen ?
<s> How is this story different ? </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Im Raum war ein junger Mann , <unk> .
<s> One of the young men in the room was <unk> . </s> <pad> <pad> <pad> <pad> <pad>
Ich brachte rund 90 junge <unk> Führungskräfte zusammen .
<s> I brought together about 90 young Somali leaders . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Es vergeht vielleicht ein Jahr , aber nichts .
<s> Maybe a year passes , nothing . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Sein Dorf liegt in der Nähe von <unk> .
<s> His village is near <unk> . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Ich d

In [113]:
# evaluator = NMTEvaluator([bs_encoder, bs_decoder], DE, EN, reverse_enc_input=False)
evaluator = NMTEvaluator([bs_encoder, bs_decoder], DE, EN, attention=True,
                        record_attention=True)
print(evaluator.evaluate(test_iter))
print(evaluator.evaluate(val_iter))
test_iter.init_epoch()
debug_iter = iter(test_iter)
for i in range(10):
    batch = next(debug_iter)
debug_set = [batch.src.data[:, i] for i in range(batch.src.data.size(1))]
debug_ans = [batch.trg.data[:, i] for i in range(batch.trg.data.size(1))]
evaluator.predict(pred_set, fn='predictions.txt',beam_size=100)

Target padding token: 1
Using CUDA...
Validation time: 1.223257 seconds
6.50678771148015
Validation time: 0.874000 seconds
5.119931039703331
torch.Size([1156000])
torch.Size([1156000])
torch.Size([100, 1])
torch.Size([4, 100, 500])
torch.Size([100, 12, 500])


RuntimeError: dimension out of range (expected to be in range of [-2, 1], but got 2)

In [79]:
train_iter, val_iter = data.BucketIterator.splits((train, val), batch_size=32, device=-1,
                                                  repeat=False, sort_key=lambda x: len(x.src))
bs_encoder = BaseEncoder(DE, hidden_size=500, num_layers=4, word_features=500)
bs_decoder = BaseDecoder(EN, hidden_size=500, num_layers=4, word_features=500)
trainer = NMTTrainer([bs_encoder, bs_decoder], DE, EN, lrn_rate=0.7, 
                     lrn_decay='adaptive', reverse_enc_input=False)
evaluator = NMTEvaluator([bs_encoder, bs_decoder], DE, EN, reverse_enc_input=False)
trainer.train(train_iter, verbose=True, le=evaluator, val_iter=val_iter)

Target padding token: 1
Using CUDA...
Target padding token: 1
Using CUDA...
Epoch 0, loss: 75.106003, norm: 5.068851, elapsed: 601.591950, lrn_rate: 0.700000
Validation time: 1.177268 seconds
Validation set metric: 15.046797
Epoch 1, loss: 64.709015, norm: 6.823921, elapsed: 1202.462563, lrn_rate: 0.700000
Validation time: 1.123111 seconds
Validation set metric: 10.318951
Epoch 2, loss: 55.162987, norm: 5.810015, elapsed: 1815.646121, lrn_rate: 0.700000
Validation time: 1.158648 seconds
Validation set metric: 8.200983
Epoch 3, loss: 48.013638, norm: 6.298200, elapsed: 2430.358610, lrn_rate: 0.700000
Validation time: 1.222892 seconds
Validation set metric: 7.256750
Epoch 4, loss: 55.855797, norm: 8.171973, elapsed: 3034.166331, lrn_rate: 0.700000
Validation time: 1.161787 seconds
Validation set metric: 7.055706
Epoch 5, loss: 51.494362, norm: 9.309718, elapsed: 3637.575163, lrn_rate: 0.700000
Validation time: 1.184436 seconds
Validation set metric: 7.108363
Decaying LR to 0.350000
Epoch

KeyboardInterrupt: 

In [110]:
train_iter, val_iter = data.BucketIterator.splits((train, val), batch_size=32, device=-1,
                                                  repeat=False, sort_key=lambda x: len(x.src))
bs_encoder = BaseEncoder(DE, hidden_size=500, num_layers=4, word_features=500, dropout=0.2)
# TODO: decide whether to add dropout to output states of encoder!
bs_decoder = AttnDecoder(EN, hidden_size=500, num_layers=4, word_features=500, dropout=0.2)
trainer = NMTTrainer([bs_encoder, bs_decoder], DE, EN, lrn_rate=1.0, 
                     lrn_decay='adaptive', attention=True)
evaluator = NMTEvaluator([bs_encoder, bs_decoder], DE, EN, attention=True,
                        record_attention=True)
trainer.train(train_iter, verbose=True, le=evaluator, val_iter=val_iter)

Target padding token: 1
Using CUDA...
Target padding token: 1
Using CUDA...
Epoch 0, loss: 83.176491, norm: 3.746789, elapsed: 515.477625, lrn_rate: 1.000000
Validation time: 0.759316 seconds
Validation set metric: 19.221707
Epoch 1, loss: 72.469406, norm: 5.242597, elapsed: 1034.874019, lrn_rate: 1.000000
Validation time: 0.888308 seconds
Validation set metric: 13.506300
Epoch 2, loss: 58.292755, norm: 5.588605, elapsed: 1554.866436, lrn_rate: 1.000000
Validation time: 0.862921 seconds
Validation set metric: 8.196184
Epoch 3, loss: 51.758568, norm: 5.508223, elapsed: 2074.353652, lrn_rate: 1.000000
Validation time: 0.833768 seconds
Validation set metric: 6.917161
Epoch 4, loss: 56.023895, norm: 7.027678, elapsed: 2593.641258, lrn_rate: 1.000000
Validation time: 0.888862 seconds
Validation set metric: 6.134933
Epoch 5, loss: 55.090607, norm: 7.383235, elapsed: 3114.329509, lrn_rate: 1.000000
Validation time: 1.042574 seconds
Validation set metric: 5.983735
Epoch 6, loss: 44.098995, nor

Validation time: 0.868033 seconds
Validation set metric: 5.119931
Decaying LR to 0.000000
Epoch 50, loss: 25.068045, norm: 8.173462, elapsed: 26487.876106, lrn_rate: 0.000000
Validation time: 0.804122 seconds
Validation set metric: 5.119931
Epoch 51, loss: 23.197910, norm: 6.603655, elapsed: 27007.840163, lrn_rate: 0.000000
Validation time: 0.846102 seconds
Validation set metric: 5.119931
Decaying LR to 0.000000
Epoch 52, loss: 26.191601, norm: 6.955893, elapsed: 27526.307991, lrn_rate: 0.000000
Validation time: 0.977962 seconds
Validation set metric: 5.119931
Epoch 53, loss: 22.765911, norm: 6.720936, elapsed: 28044.532676, lrn_rate: 0.000000
Validation time: 0.966261 seconds
Validation set metric: 5.119931
Decaying LR to 0.000000
Epoch 54, loss: 26.660145, norm: 7.047979, elapsed: 28563.961328, lrn_rate: 0.000000
Validation time: 0.835158 seconds
Validation set metric: 5.119931
Epoch 55, loss: 26.209167, norm: 7.166502, elapsed: 29081.288782, lrn_rate: 0.000000
Validation time: 0.868

KeyboardInterrupt: 

In [39]:
print(evaluator.evaluate(val_iter))
print(len(train_iter))
print(evaluator.attns_log[0][4])
val_iter.init_epoch()
batch = next(iter(val_iter))
sent_inspect(batch, 4)

Validation time: 0.456792 seconds
6.048956472651303
3722
Variable containing:
 0.6505  0.1110  0.0659  0.1725
 0.5297  0.1554  0.2806  0.0343
 0.0089  0.4239  0.4734  0.0938
 0.0180  0.0371  0.9122  0.0327
 0.0822  0.0702  0.3141  0.5335
 0.4222  0.2591  0.0997  0.2190
 0.1612  0.3041  0.4915  0.0432
 0.2962  0.4637  0.0781  0.1620
[torch.cuda.FloatTensor of size 8x4 (GPU 0)]

Source
Ich hatte Angst –
Target
<s> And I was scared . </s> <pad> <pad>


800
[20, 35, 0, 8, 4811, 0, 3351, 14, 9129]


In [127]:
print(EN.vocab.stoi['<s>'])
print(EN.vocab.itos[2])

2
<s>


## OLD STUFF BELOW HERE